# NeMo Models

In [ ]:
# Import NeMo and ASR collection
import nemo
import nemo.collections.asr as nemo_asr
try:
    nf = nemo.core.NeuralModuleFactory()
except:
    print("GPU was not detected. Running on CPU")
    nf = nemo.core.NeuralModuleFactory(placement=nemo.core.DeviceType.CPU)    

## NeMoModel instantiation without pre-trained weights

A *NeMoModel* is a kind of NeuralModule which contains other neural modules inside it.
NeMoModel can have other NeuralModules inside and their mode, and topology of connections can
depend on the mode in which NeMo model is used (training or evaluation).

### Because NeMoModel is a NeuralModule, regular constructor-based initialization applies

In [ ]:
#First, load the config from YAML file
from ruamel.yaml import YAML
yaml = YAML(typ="safe")
with open("../configs/jasper_an4.yaml") as file:
    model_definition = yaml.load(file)

In [ ]:
quartznet_model1 = nemo.collections.asr.models.QuartzNet(
    preprocessor_params=model_definition['AudioToMelSpectrogramPreprocessor'],
    encoder_params=model_definition['JasperEncoder'],
    decoder_params=model_definition['JasperDecoderForCTC'])
print(f"Created QuartzNet model with {quartznet_model1.num_weights} weights")

### Because NeMoModel is a NeuralModule, regular config import/export work

In [ ]:
quartznet_model1.export_to_config("qn1.yaml")

In [ ]:
quartznet_model2 = nemo.collections.asr.models.QuartzNet.import_from_config(config_file="qn1.yaml")
print(f"Created QuartzNet model with {quartznet_model2.num_weights} weights")

## NeMoModel instantiation with pre-trained weights

In [ ]:
# List all available models from NGC
for checkpoint in nemo.collections.asr.models.ASRConvCTCModel.list_pretrained_models():
    print(checkpoint.pretrained_model_name)

In [ ]:
# Automagically go to NGC and instantiate a model and weights
quartznet_model3 = nemo_asr.models.QuartzNet.from_pretrained(model_info="QuartzNet15x5-En")
print(f"Created QuartzNet model with {quartznet_model3.num_weights} weights")

# Export model to ".nemo" format

## Export to ".nemo" file - all params, structure and weights

In [ ]:
quartznet_model3.save_to('quartznet.nemo')

In [ ]:
quartznet_model4 = nemo_asr.models.QuartzNet.from_pretrained(model_info='quartznet.nemo')

In [ ]:
# ".nemo" file is just an arxiv with all of the model's details and weights
! mv quartznet.nemo quartznet.tar.gz
! tar -xvf quartznet.tar.gz

## Export to ".nemo" file - for deployment with NVIDIA Jarvis

In [ ]:
quartznet_model3.save_to('quartznet_for_Jarvis.nemo', optimize_for_deployment=True)

In [ ]:
# ".nemo" file optimized for deployment will only contain eval structure and .onnx files
! mv quartznet_for_Jarvis.nemo quartznet_for_Jarvis.nemo.tar.gz
! tar -xvf quartznet_for_Jarvis.nemo.tar.gz

## NeMoModels can be used just as any other Neural Module

In [ ]:
# Change these to point to your training data
train_manifest = "/Users/okuchaiev/Data/an4_dataset/an4_train.json"
val_manifest = "/Users/okuchaiev/Data/an4_dataset/an4_val.json"
labels = model_definition['labels']
data_layer = nemo_asr.AudioToTextDataLayer(manifest_filepath=train_manifest, labels=labels, batch_size=16)
data_layerE = nemo_asr.AudioToTextDataLayer(manifest_filepath=val_manifest, labels=labels, batch_size=16)
ctc_loss = nemo_asr.CTCLossNM(num_classes=len(labels))
greedy_decoder = nemo_asr.GreedyCTCDecoder()

In [ ]:
audio_signal, audio_signal_len, transcript, transcript_len = data_layer()
log_probs, encoded_len = quartznet_model4(input_signal=audio_signal, length=audio_signal_len)
predictions = greedy_decoder(log_probs=log_probs)
loss = ctc_loss(log_probs=log_probs, targets=transcript,
                input_length=encoded_len, target_length=transcript_len)

In [ ]:
# START TRAINING 
tensors_to_evaluate=[predictions, transcript, transcript_len]
from functools import partial
from nemo.collections.asr.helpers import monitor_asr_train_progress
train_callback = nemo.core.SimpleLossLoggerCallback(
    tensors=[loss]+tensors_to_evaluate,
    print_func=partial(monitor_asr_train_progress, labels=labels))
nf.train(tensors_to_optimize=[loss],
                callbacks=[train_callback],
                optimizer="novograd",
                optimization_params={"num_epochs": 30, "lr": 1e-2,
                                    "weight_decay": 1e-3})